In [1]:
import sys
sys.path.insert(1, '../..')

import pandas as pd
import numpy as np

In [2]:
data = pd.read_csv("../../data/processed/twitter16_dataset_with_tvt.csv", lineterminator="\n")
data.head()

tweet_id                                         tweet_text  \
0  656955120626880512  correct predictions in back to the future ii U...   
1  615689290706595840  .@whitehouse in rainbow colors for #scotusmarr...   
2  613404935003217920  cops bought the alleged church shooter burger ...   
3  731166399389962242  🔥ca kkk grand wizard 🔥 endorses @hillaryclinto...   
4  714598641827246081  an open letter to trump voters from his top st...   

        label       tvt  cv_fold        tt        tvt2      tvt2_1    tvt2_2  \
0       false  training        1  training  validation    training  training   
1        true  training        3  training    training  validation  training   
2       false  training        2      test    training    training  training   
3  unverified  training        3      test    training    training  training   
4  unverified  training        1      test    training  validation  training   

       tvt2_3  
0  validation  
1    training  
2    training  
3    training  
4    training

In [3]:
label_rnr = []
for i, d in data.iterrows():
    if d['label'] in ["unverified", "true", "false"]:
        label_rnr.append("rumours")
    else:
        label_rnr.append("non-rumours")
        
data['label_rnr'] = pd.Series(label_rnr)
data.head()

tweet_id                                         tweet_text  \
0  656955120626880512  correct predictions in back to the future ii U...   
1  615689290706595840  .@whitehouse in rainbow colors for #scotusmarr...   
2  613404935003217920  cops bought the alleged church shooter burger ...   
3  731166399389962242  🔥ca kkk grand wizard 🔥 endorses @hillaryclinto...   
4  714598641827246081  an open letter to trump voters from his top st...   

        label       tvt  cv_fold        tt        tvt2      tvt2_1    tvt2_2  \
0       false  training        1  training  validation    training  training   
1        true  training        3  training    training  validation  training   
2       false  training        2      test    training    training  training   
3  unverified  training        3      test    training    training  training   
4  unverified  training        1      test    training  validation  training   

       tvt2_3 label_rnr  
0  validation   rumours  
1    training   rumours  
2    training   rumours  
3    training   rumours  
4    training   rumours

In [4]:
# labels_str = ['rumour', 'non-rumour']
label_type = "label"
labels_str = data[label_type].unique().tolist()
labels_str

['false', 'true', 'unverified', 'non-rumor']

In [5]:
labels = []
for i, d in data.iterrows():
    lab = labels_str.index(d[label_type])
    labels.append(lab)
labels[:10]

[0, 1, 0, 2, 2, 1, 2, 3, 3, 0]

In [6]:
import string
import nltk
from nltk.tokenize import TweetTokenizer

tokenizer = TweetTokenizer(reduce_len=True)


def text2unigrams(text):
    texts = tokenizer.tokenize(text.encode('ascii', 'ignore').decode('utf8'))
    texts = [t for t in texts if t not in string.punctuation]
    texts = [t for t in texts if t not in ['URL', '‘', '’']]
    
    unigrams = texts
    
    return unigrams


def text2bigrams(text):
    texts = tokenizer.tokenize(text.encode('ascii', 'ignore').decode('utf8'))
    texts = [t for t in texts if t not in string.punctuation]
    texts = [t for t in texts if t not in ['URL', '‘', '’']]
    
    bigrams = nltk.bigrams(texts)
    bigrams = map(' '.join, bigrams)
    bigrams = [bgr for bgr in bigrams]
    
    return bigrams


def text2trigrams(text):
    texts = tokenizer.tokenize(text.encode('ascii', 'ignore').decode('utf8'))
    texts = [t for t in texts if t not in string.punctuation]
    texts = [t for t in texts if t not in ['URL', '‘', '’']]
    
    trigrams = nltk.trigrams(texts)
    trigrams = map(' '.join, trigrams)
    trigrams = [bgr for bgr in trigrams]
    
    return trigrams


def bigrams_vectors_generation(texts):
    bigram_vectors = []
    for text in texts:
        bigrams = text2bigrams(text)

        init_vec = [0.0 for _ in range(len(bigram_vector_base) + 1)]
        for bgr in bigrams:
            if bgr in bigram_vector_base:
                idx = bigram_vector_base.index(bgr)
                init_vec[idx] = 1.0
            else:
                init_vec[-1] = 1.0
        bigram_vectors.append(init_vec)
    
    return bigram_vectors


def custom_vectors_generation(texts, vector_terms):
    vectors = []
    for text in texts:
        bigrams = text2bigrams(text)
        trigrams = text2trigrams(text)

        init_vec = [0.0 for _ in range(len(vector_terms) + 1)]
        for bgr in bigrams:
            if bgr in vector_terms:
                idx = vector_terms.index(bgr)
                init_vec[idx] = 1.0
            else:
                init_vec[-1] = 1.0
        for tgr in trigrams:
            if tgr in vector_terms:
                idx = vector_terms.index(tgr)
                init_vec[idx] = 1.0
            else:
                init_vec[-1] = 1.0
        vectors.append(init_vec)
    
    return vectors

In [7]:
texts = data['tweet_text'].tolist()
# vectors = bigrams_vectors_generation(texts)

In [8]:
import nltk
from nltk.collocations import *

top_n = 2000
bigram_measures = nltk.collocations.BigramAssocMeasures()
trigram_measures = nltk.collocations.TrigramAssocMeasures()

finder2 = BigramCollocationFinder.from_words([])
finder3 = TrigramCollocationFinder.from_words([])

# generating bigram and trigram
for text in texts:
    unigrams = text2unigrams(text)
    bigrams = text2bigrams(text)
    trigrams = text2trigrams(text)
    
    for ngrm in unigrams:
        if ngrm not in finder2.word_fd:
            finder2.word_fd[ngrm] = 0
        finder2.word_fd[ngrm] += 1
        finder2.N += 1
        
        if ngrm not in finder3.word_fd:
            finder3.word_fd[ngrm] = 0
        finder3.word_fd[ngrm] += 1
        finder3.N += 1
    
    for ngrm in bigrams:
        term = tuple([i for i in ngrm.split()])
        
        if term not in finder2.ngram_fd:
            finder2.ngram_fd[term] = 0
            
        finder2.ngram_fd[term] += 1

    for ngrm in trigrams:
        term = tuple([i for i in ngrm.split()])
        
        if term not in finder3.ngram_fd:
            finder3.ngram_fd[term] = 0
            
        finder3.ngram_fd[term] += 1
        
# only bigrams that appear 3+ times
finder2.apply_freq_filter(3)
finder3.apply_freq_filter(3)

combined = []
for res in finder2.score_ngrams(bigram_measures.pmi):
    combined.append(res)
for res in finder3.score_ngrams(trigram_measures.pmi):
    combined.append(res)
combined = sorted(combined, key=lambda x: x[1], reverse=True)

In [9]:
print(len(finder2.ngram_fd), len(finder3.ngram_fd))
count = 0
for k, v in combined:
    print(f"{k} - {v}")
    count += 1
    if count >= 10:
        break

501 275
('1991', 'book', 'quotes') - 22.928836235477128
('kissing', 'islands', 'greenland') - 22.928836235477128
('confiscates', 'several', 'thousand') - 22.758911234034812
('fda', 'confiscates', 'several') - 22.758911234034812
('niagara', 'falls', 'turned') - 22.606908140589766
('several', 'thousand', 'chickens') - 22.34387373475597
('drive', 'destruction', 'company') - 21.86994254642356
('cpl', 'nathan', 'cirillo') - 21.536518812698368
('blacks', '1991', 'book') - 21.513798736198282
('signers', 'stock', 'sinks') - 21.469404616839828


In [10]:
term_vector_base = [" ".join(c[0]) for c in combined[:top_n]]
vectors = custom_vectors_generation(texts, term_vector_base)

In [11]:
len(vectors[102])

777

In [12]:
min_score = 100
max_score = 0
n_2gram = 0
n_3gram = 0
for k, v in combined[:top_n]:
    min_score = min(min_score, v)
    max_score = max(max_score, v)
    
    if len(k) == 2:
        n_2gram += 1
    
    if len(k) == 3:
        n_3gram += 1
        
print(min_score)
print(max_score)
print(n_2gram)
print(n_3gram)

0.6362816335444546
22.928836235477128
501
275


In [13]:
train_vectors = np.array([vectors[i] for i, d in data.iterrows() if d['tvt2'] == 'training'])
val_vectors = np.array([vectors[i] for i, d in data.iterrows() if d['tvt2'] == 'validation'])
test_vectors = np.array([vectors[i] for i, d in data.iterrows() if d['tvt2'] == 'testting'])

train_labels = np.array([labels[i] for i, d in data.iterrows() if d['tvt2'] == 'training'])
val_labels = np.array([labels[i] for i, d in data.iterrows() if d['tvt2'] == 'validation'])
test_labels = np.array([labels[i] for i, d in data.iterrows() if d['tvt2'] == 'testting'])

In [14]:
from sklearn.svm import LinearSVC

from library.classification import SKLearnClassification
from library.evaluation import ConfusionMatrix

svm = LinearSVC()

model = SKLearnClassification(svm, "Support Vector Machine")
print(f"\n--- START ---")
model.train(train_vectors, train_labels, "Twitter16")

print("Validation Set")
preds = model.predict(val_vectors)

conf_mat = ConfusionMatrix(
    labels=np.array([[1 if j == v else 0 for j in range(len(labels_str))] for v in val_labels]),
    predictions=np.array([[1 if j == p else 0 for j in range(len(labels_str))] for p in preds]),
    binary=False
)
conf_mat.evaluate(labels_str)

print("--- END ---\n")


--- START ---
---> execution time : 0.01 seconds
Validation Set
192 vs 192
Multi Class Evaluation

Class false Evaluation
- Precision : 75.0 %
- Recall : 65.217 %
- F1 : 0.69767

Class true Evaluation
- Precision : 78.431 %
- Recall : 72.727 %
- F1 : 0.75472

Class unverified Evaluation
- Precision : 83.871 %
- Recall : 56.522 %
- F1 : 0.67532

Class non-rumor Evaluation
- Precision : 48.571 %
- Recall : 75.556 %
- F1 : 0.5913

Combined Evaluation
- Accuracy : 67.708 %
- Precision : 71.468 %
- Recall : 67.505 %
- F1 : 0.6943

- Average Confidence : 100.0 %
Model, Combined,,,,false,,,true,,,unverified,,,non-rumor,,,
Anonymous, 67.708, 71.468, 67.505, 0.6943, 75.0, 65.217, 0.69767, 78.431, 72.727, 0.75472, 83.871, 56.522, 0.67532, 48.571, 75.556, 0.5913, 
--- END ---



In [15]:
from scipy.sparse import csr_matrix

coefs = model.model.coef_
if type(coefs) == csr_matrix:
    coefs.toarray().tolist()[0]
else:
    coefs.tolist()
coefs_and_features = list(zip(coefs[0], term_vector_base))

# Most predictive overall
# coefs_and_features = sorted(coefs_and_features, key=lambda x: x[0], reverse=True)
coefs_and_features = sorted(coefs_and_features, key=lambda x: abs(x[0]), reverse=True)
print(f"Total tokens : {len(coefs_and_features)}")

Total tokens : 776


In [16]:
coefs.shape

(4, 777)

In [17]:
coefs_and_features[:10]

[(1.4564242351941268, 'el chapo'),
 (1.4564242075211555, '#opkkk #hoodsoff'),
 (1.4065789857626652, 'mass shootings'),
 (1.3107817682706258, 'red cross'),
 (1.227858430496503, 'burger king'),
 (1.128577166985884, 'the future'),
 (1.0968672625996452, 'new cnn'),
 (1.0923181104601813, 'new orleans'),
 (1.0923180613055532, 'poll @hillaryclinton'),
 (1.0776436428644736, 'that he')]

In [18]:
coefs_and_features[-10:]

[(0.0, 'who have'),
 (0.0, 'is now'),
 (0.0, 'escaped the'),
 (0.0, 'the sports'),
 (0.0, 'on donald'),
 (0.0, 'house is'),
 (0.0, 'hostages in'),
 (0.0, 'in shooting'),
 (0.0, 'of this'),
 (0.0, 'at a')]

In [20]:
n_best = 750
best_tokens = []

# for cf in coefs_and_features[-n_best:]:
#     best_tokens.append(cf[1])

for cf in coefs_and_features[:n_best]:
    best_tokens.append(cf[1])
    
print(best_tokens[:10])
    
with open("../../data/processed/twitter16-multi_best_terms.txt", "w") as f:
    for token in best_tokens:
        f.write(token + "\n")

['el chapo', '#opkkk #hoodsoff', 'mass shootings', 'red cross', 'burger king', 'the future', 'new cnn', 'new orleans', 'poll @hillaryclinton', 'that he']
